In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import neweggutils
import re

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('final.csv')

In [4]:
df.drop(labels=['Unnamed: 0'], inplace=True, axis=1)

In [6]:
df_clean = df.copy()

# Drop duplicates and unnecessary columns

In [9]:
df_clean = df_clean[~df_clean.duplicated()]

In [12]:
df_clean.columns.values

array(['price', 'link', 'Brand', 'Series', 'Model', 'Part Number', 'Type',
       'Form Factor', 'Usage', 'Colors', 'Processor',
       'Processor Main Features', 'Cache Per Processor', 'Memory',
       'Storage', 'Optical Drive', 'Graphics', 'Power Supply',
       'Operating System', 'CPU Type', 'CPU Speed', 'L3 Cache Per CPU',
       'CPU Main Features', 'GPU/VGA Type', 'Video Memory',
       'Virtual Reality Ready', 'Memory Capacity', 'Memory Speed',
       'Memory Spec', 'Memory Slot (Total)', 'Maximum Memory Supported',
       'Optane Memory', 'HDD', 'HDD RPM', 'Optical Drive Type',
       'Screen Size', 'LAN Speed', 'WLAN', 'Bluetooth', 'Mouse Type',
       'Keyboard Type', 'Dimensions (H x W x D)', 'Weight',
       'Graphics Interface', 'SSD', 'Keyboard', 'Storage Spec',
       'Optical Drive Spec', 'Media Slots', 'Display Feature',
       'Front USB', 'Mouse', 'Special Features', 'Package Contents',
       'Software Included', 'Audio Chipset', 'LAN Chipset', 'Video Ports',
    

In [15]:
df_clean = df_clean.drop(labels=['Series', 'Model', 'Part Number', 'Model', 'Colors', 'Cache Per Processor',
                                 'Optical Drive', 'L3 Cache Per CPU', 'Virtual Reality Ready', 'Optane Memory',
                                 'LAN Speed', 'WLAN', 'Bluetooth', 'Mouse Type', 'Keyboard Type',
                                 'Dimensions (H x W x D)', 'Keyboard', 'Mouse', 'Special Features',
                                 'Package Contents', 'Software Included', 'Audio Chipset', 'LAN Chipset',
                                 'Speaker Configuration', 'Front Audio Ports', 'L2 Cache Per CPU', 
                                 'COM', 'Speaker Features', 'FIR', 'Card Reader', 'Installed Qty', 'Touchscreen',
                                 'LPT', 'External Bays', 'Internal Bays', 'Package Type', 'Optical Drive 2',
                                 'Optical Drive 2 Type', 'Front IEEE 1394'], axis=1)

In [17]:
df_clean = df_clean.drop(labels=['Power Supply', 'Memory Speed', 'Memory Spec', 'Memory Slot (Total)',
                                 'Maximum Memory Supported', 'HDD RPM', 'Weight', 'Media Slots', 'Display Feature',
                                 'HDD Interface', 'Ethernet', 'Chipset', 'Audio Features', 'Memory Slot (Available)',
                                 'Graphics Card', 'Screen Type', 'PCI Slots (Available/Total)', 'Monitor',
                                 'Audio', 'Speaker', 'SLI / Crossfire', 'Max Supported Qty'], axis=1)

In [25]:
df_clean['Processor']

,price,link,Brand,Type,Form Factor,Usage,Processor,Processor Main Features,Memory,Storage,...,SSD,Storage Spec,Optical Drive Spec,Front USB,Video Ports,Rear USB,RJ45,Rear Audio Ports,Memory Slots (Available/Total),PS/2
3196,277,https://www.newegg.com/Product/Product.aspx?It...,NaN,NaN,Slim/Small form factor,NaN,NaN,NaN,NaN,NaN,...,No,NaN,NaN,4,"1 VGA, 1 DVI",6,1 port,"1x Headphone, 1x Microphone, 1x Line In",NaN,2
3369,252,https://www.newegg.com/Product/Product.aspx?It...,HP,NaN,Slim/Small form factor,NaN,Intel Core 2 Quad Q6600 2.4 GHz,NaN,16 GB DDR3,1 TB SATA 7200 RPM,...,No,NaN,NaN,4,1 VGA,6,1 port,2 ports,NaN,2
1684,269,https://www.newegg.com/Product/Product.aspx?It...,DELL,NaN,NaN,NaN,Intel Core i5-3570 3.4 GHz,64 bit Quad-Core Processor,8 GB,2 TB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3575,181,https://www.newegg.com/Product/Product.aspx?It...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1187,345,https://www.newegg.com/Product/Product.aspx?It...,NaN,NaN,USFF,NaN,NaN,NaN,NaN,NaN,...,240 GB,NaN,NaN,2,"1 VGA, 2 DisplayPort","2 USB 2.0, 2 USB 3.0",1 port,2 ports,NaN,NaN
3609,98,https://www.newegg.com/Product/Product.aspx?It...,HP,NaN,Slim/Small form factor,NaN,Intel Core 2 Duo E7600 3.06 GHz,NaN,2 GB DDR2,80 GB SATA 7200 RPM,...,No,NaN,NaN,2,1 VGA,6,1 port,2 ports,NaN,2
1733,280,https://www.newegg.com/Product/Product.aspx?It...,DELL,Business Desktops & Workstations,Slim/Small form factor,Business,Intel Core i7-2600 3.4 GHz,NaN,8 GB,1 TB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1306,384,https://www.newegg.com/Product/Product.aspx?It...,DELL,Business Desktops & Workstations,Tower,Business,Intel Core i7-4770 3.40 GHz,NaN,8 GB,500 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3010,194,https://www.newegg.com/Product/Product.aspx?It...,HP,NaN,Slim/Small form factor,NaN,Intel Core i3-2100 3.1 GHz,NaN,4 GB DDR3,1 TB,...,NaN,NaN,NaN,2,"1 VGA, 1 Display Port",5,1 port,"1x Headphone, 1x Microphone, 1x Line In",NaN,2
3988,198,https://www.newegg.com/Product/Product.aspx?It...,DELL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
